In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
# 지도 시각화를 위해
import folium
import seaborn as sns
import plotly.express as px

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
import gspread
from google.auth import default
creds, _ = default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from google.cloud import bigquery
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(creds)

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
query_leadtime = """
SELECT
  cdate,
  way,
  avg(leadtime) as leadtime,
  count(rid) as cnt
FROM (
  SELECT
    CASE WHEN date(created_at, 'Asia/Seoul') >='2023-12-08' THEN 'After' ELSE 'Before' END as date_type,
    date(created_at, 'Asia/Seoul') as cdate,
    way,
    datetime_diff(end_at, start_at, hour) as leadtime,
    id as rid

  FROM `tianjin_replica.reservation_info`
  WHERE way IN ('d2d_round', 'd2d_oneway')
  AND date(created_at) >= "2023-11-06"
  AND state IN (1, 2, 3)
)
GROUP BY cdate, way
"""


In [ ]:
df_leadtime = pd.io.gbq.read_gbq(
    query=query_leadtime,
    project_id="socar-data"
)

In [ ]:
df_leadtime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   cdate     136 non-null    dbdate 
 1   way       136 non-null    object 
 2   leadtime  136 non-null    float64
 3   cnt       136 non-null    Int64  
dtypes: Int64(1), dbdate(1), float64(1), object(1)
memory usage: 4.5+ KB


In [ ]:
## 예약 타입에 따른 건수와 예약 리드타임 분포도
fig = px.histogram(df_leadtime, x='cdate', y='cnt', color='way',
                 title='예약 타입에 따른 건수와 예약 리드타임 분포도', barmode='group',
                 labels={'cnt': '예약건수', 'leadtime': '예약리드타임(시간)'})

fig.show()

In [ ]:
## 예약 타입에 따른 건수와 예약 리드타임 분포도
fig = px.scatter(df_leadtime, x='cnt', y='leadtime', color='way',
                 title='예약 타입에 따른 건수와 예약 리드타임 분포도',
                 labels={'cnt': '예약건수', 'leadtime': '예약리드타임(시간)'})

fig.show()

In [ ]:
## 예약타입에 따른 날짜별 리드타임 분포도
fig = px.scatter(df_leadtime, x='cdate', y='leadtime', color='way',
                 title='예약타입에 따른 날짜별 리드타임 분포도',
                 labels={'cdate': '예약 생성 날짜', 'leadtime': '예약리드타임(시간)'})

fig.show()

In [ ]:
query_click = """
WITH base AS (
	select
			region,
			-- set_region(zid) as part,
			CASE WHEN zid IN (105,9890) THEN 'air'
					 WHEN zid IN (17209) THEN 'air_infront' ELSE 'common' END as part,
      CASE WHEN way is null THEN 'click'
					 WHEN way = 'round' THEN 'round_click'
					 WHEN way IN ('d2d_round', 'd2d_oneway') THEN 'd2d_click' END as way,
			sdate,
			count(distinct(mid)) as unique_click_cnt,
			count(distinct(r.id)) as rev_cnt,
			safe_divide(count(distinct(r.id)),count(distinct(mid))) as rev_Rate,
			extract(isoweek from sdate) as w,


	from(
		select
			date(a.event_timestamp,"Asia/Seoul") as date, #존을클릭한날
			date(a.start_at, "Asia/Seoul") as sdate,#존클릭예약시작조회날
			a.member_id as mid,#존클릭예약자
			c.id as class_id, #존클릭조회차량클래스id
			date(a.start_at,"Asia/Seoul") as start_at,#예약시작일
			date(a.end_at,"Asia/Seoul") as end_at, #예약종료일


			a.zone_id as zid,        #존id
			z.zone_name as zname,
			z.region2 as region
		from socar_server_2.get_car_classes a , tianjin_replica.carzone_info z, unnest(carClasses) c

		where true=true and a.zone_id = z.id
                    AND z.id NOT IN(122, 13858, 17651, 17784)
										AND date(a.start_at, "Asia/Seoul") >= '2023-11-01'
				-- 조회시점에 따라 위 날짜 조건을 조정해야함
		) click #존클릭자수테이블

	join tianjin_replica.member_info m on m.id = click.mid and m.imaginary in (0) #멤버정보(쏘친,쏘팸)
	left join tianjin_replica.reservation_info r on r.member_id = click.mid #예약정보테이블
                                                and r.zone_id = click.zid
                                                and date(r.start_at, "Asia/Seoul") = click.sdate #예약시작과존클릭시작을leftjoin
                                                and r.state in (1,2,3) #예약,운행,완료
                                                and r.way in ('round', 'd2d_round', 'd2d_oneway') #부름, 왕복

	group by region, part, way, sdate
	having region in ('제주시','서귀포시')
	order by region, part, way, sdate
)


SELECT
		sdate, way, rev_rate
FROM base
"""

In [ ]:
df_click = pd.io.gbq.read_gbq(
    query=query_click,
    project_id="socar-data"
)

In [ ]:
df_click_d2d = df_click[df_click['way'] == 'd2d_click']
df_click_round = df_click[df_click['way'] == 'round_click']

In [ ]:
## 예약타입별 날짜에 따른 예약전환율 추이
fig = px.scatter(df_click_d2d, x='sdate', y='rev_rate', color='way',
                 title='부름예약의  날짜에 따른 예약전환율 추이',
                 labels={'sdate': '예약 날짜', 'rev_rate': '예약전환율'})

fig.show()

In [ ]:
## 예약타입별 날짜에 따른 예약전환율 추이
fig = px.scatter(df_click_round, x='sdate', y='rev_rate', color='way',
                 title='왕복예약의  날짜에 따른 예약전환율 추이',
                 labels={'sdate': '예약 날짜', 'rev_rate': '예약전환율'})

fig.show()

In [ ]:
query_idle = """
select
  date,
  CASE WHEN date BETWEEN '2023-11-06' AND '2023-12-07' THEN 'Before'
       WHEN date BETWEEN '2023-12-08' AND '2024-01-08' THEN 'After' END as date_type,

  round(sum(dp_min/(1440)),0) as total_car_cnt,
  count(case when dp_min >0 and op_min = 0 then car_id end) as idle,
  sum(op_min)/sum(dp_min) as op_rate,
  count(case when dp_min >0 and op_min = 0 then car_id end)/round(sum(dp_min/(1440)),0) as idle_rate,
  sum(dp_min)/60 as dp_hour,
  sum(op_min)/60 as op_hour,
  sum(bl_min)/60 as bl_hour,
  sum(op_min_passport_subscription)/60 as p_hour

from `socar-data.socar_biz.operation_per_car_daily_v2`
where region1 in ("제주특별자치도")
and date >= '2023-11-06' and date <= "2024-01-08"
and sharing_type in ('socar','zplus')
and zone_id not in (122,2184,11480,12072,12097,10736,10738,11947,13787,13858,14494)
group by date, date_type
"""


In [ ]:
df_idle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           64 non-null     dbdate 
 1   date_type      64 non-null     object 
 2   total_car_cnt  64 non-null     float64
 3   idle           64 non-null     Int64  
 4   op_rate        64 non-null     float64
 5   idle_rate      64 non-null     float64
 6   dp_hour        64 non-null     float64
 7   op_hour        64 non-null     float64
 8   bl_hour        64 non-null     float64
 9   p_hour         64 non-null     float64
dtypes: Int64(1), dbdate(1), float64(7), object(1)
memory usage: 5.2+ KB


In [ ]:
df_idle = pd.io.gbq.read_gbq(
    query=query_idle,
    project_id="socar-data"
)

In [ ]:
## 전/후 유휴율과 부름 건수 분포도
fig = px.histogram(df_idle, x='date', y='idle_rate', color='date_type',
                 title='전/후 유휴율과 부름 건수 분포도', barmode='group',
                 labels={'idle_rate': '제주 전체 유휴율', 'nuse': '부름 건수'},
                 hover_data=['idle_rate'],
                 histfunc='avg')

# y축의 범위를 0.0에서 1.0으로 설정
fig.update_layout(yaxis=dict(range=[0.0, 0.2]))


fig.show()

In [ ]:
# Calculate average idle_rate for each date_type
avg_idle_rate = df_idle.groupby(['date', 'date_type'])['idle_rate'].mean().reset_index()

# 전/후 유휴율과 부름 건수 평균 분포도 as a line graph
fig = px.line(avg_idle_rate, x='date', y='idle_rate', color='date_type',
              title='전/후 유휴율과 부름 건수 평균 분포도',
              labels={'idle_rate': '제주 전체 평균 유휴율', 'nuse': '부름 건수'})

# y축의 범위를 0.0에서 0.2으로 설정
fig.update_layout(yaxis=dict(range=[0.0, 0.3]))

fig.show()

In [ ]:
query_vroom = """
SELECT
  region2,
  CASE WHEN date BETWEEN '2023-11-06' AND '2023-12-07' THEN 'BEFORE' WHEN date BETWEEN '2023-12-08' AND '2024-01-08' THEN 'AFTER' END as date_type,
  reservation_id as rid,
  way,
  reservation_created_lng as c_lng,
  reservation_created_lat as c_lat

FROM `soda_store.reservation_v2`
WHERE date BETWEEN '2023-11-06' AND '2024-01-08'
AND region1 IN ('제주특별자치도')
AND way IN ('d2d_round', 'd2d_oneway')
AND reservation_created_lng is not null
"""

In [ ]:
df_vroom = pd.io.gbq.read_gbq(
    query=query_vroom,
    project_id="socar-data"
)

In [ ]:
!pip install folium --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: folium
    Found existing installation: folium 0.14.0
    Uninstalling folium-0.14.0:
      Successfully uninstalled folium-0.14.0


In [ ]:
# 지도의 초기 위치 설정
# map_center = [df_map['zone_lat'].mean(), df_map['zone_lng'].mean()]
map_zoom = 12

# Attribution for OpenStreetMap tiles
attribution = 'Map data © OpenStreetMap contributors'

# Folium 지도 생성 중심점 바꿔야되는데 귀찮아서 일단 고.
# map_object = folium.Map(location=[33.504178, 126.494976], zoom_start=map_zoom, tiles='Stamen Terrain', attr=attribution)
map_object = folium.Map(location=[33.504178, 126.494976], zoom_start=map_zoom, tiles='OpenStreetMap', attr=attribution)
# 각 행에 대해 지도에 핀 추가
for index, row in df_vroom.iterrows():
    # 핀에 표시될 정보
    popup_text = f"rid: {row['rid']}, date_type: {row['date_type']}"

    # 지도에 핀 추가
    folium.Marker(location=[row['c_lat'], row['c_lng']], popup=popup_text).add_to(map_object)

# 지도 출력
map_object

In [ ]:
from folium.plugins import HeatMap

# zone_lat, zone_lng의 고유값들의 평균 계산
avg_zone_lat = df_vroom['c_lat'].drop_duplicates().mean()
avg_zone_lng = df_vroom['c_lng'].drop_duplicates().mean()

# 지도의 초기 위치와 줌 설정
map = folium.Map(location=[avg_zone_lat, avg_zone_lng], zoom_start=12)

# 히트맵 데이터 준비
heat_data = [[row['c_lat'], row['c_lng']] for index, row in df_vroom.iterrows()]

# 히트맵 추가
HeatMap(heat_data).add_to(map)

# 지도 표시
map

In [ ]:
query_car = """
WITH base AS (
  SELECT
      extract(year from date) as year
      , extract(month from date) as month
      , r.isoweek as isoweek
      , r.date as date
      , CASE WHEN  extract(month FROM date) IN (1, 2, 3) THEN '1Q'
            WHEN  extract(month FROM date) IN (4, 5, 6) THEN '2Q'
            WHEN  extract(month FROM date) IN (7, 8, 9) THEN '3Q' ELSE '4Q' END as q
      , r.way as way
      , case when r.way IN ("d2d_round", "d2d_oneway") then 'd2d'
            when r.way = "z2d_oneway" then "z2d_oneway" else 'round' end as way_g
      , case when date BETWEEN '2023-11-06' AND '2023-12-07' THEN 'Before'
             WHEN date BETWEEN '2023-12-08' ANd '2024-01-08' THEN 'After' END as date_type

      , r.region1
      , r.region2

      , cl.car_model

      , count(r.reservation_id) as nuse -- 예약건수
      , sum(r.utime) as utime -- 이용시간

      , sum(revenue) as revenue
      , sum(_rev_d2d) as _rev_d2d -- 부름매출(부름요금 + 편도요금)

      , sum(contribution_margin) as margin

      , sum(__rev_d2d_fee) as d2d_fee -- 부름 요금
      , sum(__rev_d2d_return_fee) as d2d_return_fee -- 부름편도 요금

      , sum(parking_cost_d2d) as parking_cost_d2d --주차료 _부름
      , sum(transport_cost_d2d) as transport_cost_d2d --운반비(부름)

  FROM `socar-data.soda_store.reservation_v2` r
  LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
  LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
  LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
  LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
  WHERE 1=1 AND date >= '2023-11-06'
            AND r.member_imaginary IN (0,9)
            AND region1 = '제주특별자치도'
          --  AND r.zone_id NOT IN (105, 9890, 17209)
  group by isoweek, date, way, way_g, month, region1, region2, car_model, date_type
  -- order by isoweek, date, way, way_g, month, region1, region2, car_model
)

SELECT
  date_type,
  way, car_model,
  sum(nuse) as nuse,
  sum(utime)/sum(nuse) as use_time,
  sum(margin) as margin,
  sum(margin)/sum(nuse) as use_margin,

  sum(transport_cost_d2d) as transport_cost_d2d,
  sum(transport_cost_d2d)/sum(nuse) as use_transport_cost
FROM base
WHERE date_type is not null
AND way IN ('d2d_round', 'd2d_oneway')
GROUP BY date_type, way, car_model
ORDER BY date_type, car_model, way
"""

In [ ]:
df_car = pd.io.gbq.read_gbq(
    query=query_car,
    project_id="socar-data"
)

In [ ]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date_type           23 non-null     object 
 1   way                 23 non-null     object 
 2   car_model           23 non-null     object 
 3   nuse                23 non-null     Int64  
 4   use_time            23 non-null     float64
 5   margin              23 non-null     float64
 6   use_margin          23 non-null     float64
 7   transport_cost_d2d  23 non-null     float64
 8   use_transport_cost  23 non-null     float64
dtypes: Int64(1), float64(5), object(3)
memory usage: 1.8+ KB


In [ ]:
# import plotly.express as px

# df = px.data.gapminder().query("country == 'Canada'")
# fig = px.bar(df, x='year', y='pop',
#              hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
#              labels={'pop':'population of Canada'}, height=400)
# fig.show()

# import plotly.express as px
# df = px.data.tips()
# fig = px.histogram(df, x="sex", y="total_bill",
#              color='smoker', barmode='group',
#              height=400)
# fig.show()

In [ ]:
# Plotly Express를 사용하여 막대 그래프 생성
fig = px.histogram(df_car, x='car_model', y='use_margin',
                   color='date_type', barmode='group',
                   hover_data=['use_margin', 'use_time'],
             title='차량별 건당 공헌이익')

# 그래프 표시
fig.show()

In [ ]:
# Plotly Express를 사용하여 막대 그래프 생성
fig = px.histogram(df_car, x='way', y='use_margin',
                   color='date_type', barmode='group',
                   hover_data=['use_margin', 'use_time'],
             title='way별 건당 공헌이익')

# 그래프 표시
fig.show()

In [ ]:
# Plotly Express를 사용하여 막대 그래프 생성
fig = px.histogram(df_car, x='way', y='nuse',
                   color='date_type', barmode='group',
                   hover_data=['nuse', 'use_time'],
             title='way별 건수')

# 그래프 표시
fig.show()

In [ ]:
query_car_margin = """
WITH base AS (
  SELECT
      extract(year from date) as year
      , extract(month from date) as month
      , r.isoweek as isoweek
      , r.date as date
      , CASE WHEN  extract(month FROM date) IN (1, 2, 3) THEN '1Q'
            WHEN  extract(month FROM date) IN (4, 5, 6) THEN '2Q'
            WHEN  extract(month FROM date) IN (7, 8, 9) THEN '3Q' ELSE '4Q' END as q
      , r.way as way
      , case when r.way IN ("d2d_round", "d2d_oneway") then 'd2d'
            when r.way = "z2d_oneway" then "z2d_oneway" else 'round' end as way_g
      , case when date BETWEEN '2023-11-06' AND '2023-12-07' THEN 'Before'
             WHEN date BETWEEN '2023-12-08' ANd '2024-01-08' THEN 'After' END as date_type

      , r.region1
      , r.region2

      , cl.car_model

      , count(r.reservation_id) as nuse -- 예약건수
      , sum(r.utime) as utime -- 이용시간

      , sum(revenue) as revenue
      , sum(_rev_d2d) as _rev_d2d -- 부름매출(부름요금 + 편도요금)

      , sum(contribution_margin) as margin

      , sum(__rev_d2d_fee) as d2d_fee -- 부름 요금
      , sum(__rev_d2d_return_fee) as d2d_return_fee -- 부름편도 요금

      , sum(parking_cost_d2d) as parking_cost_d2d --주차료 _부름
      , sum(transport_cost_d2d) as transport_cost_d2d --운반비(부름)

  FROM `socar-data.soda_store.reservation_v2` r
  LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
  LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
  LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
  LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
  WHERE 1=1 AND date >= '2023-11-06'
            AND r.member_imaginary IN (0,9)
            AND region1 = '제주특별자치도'
          --  AND r.zone_id NOT IN (105, 9890, 17209)
  group by isoweek, date, way, way_g, month, region1, region2, car_model, date_type
  -- order by isoweek, date, way, way_g, month, region1, region2, car_model
)

SELECT
  date_type,
  way, car_model,
  region2,
  sum(nuse) as nuse,
  sum(utime)/sum(nuse) as use_time,
  sum(margin) as margin,
  sum(margin)/sum(nuse) as use_margin,

  sum(transport_cost_d2d) as transport_cost_d2d,
  sum(transport_cost_d2d)/sum(nuse) as use_transport_cost
FROM base
WHERE date_type = "After"
-- AND way IN ('d2d_round', 'd2d_oneway')
GROUP BY date_type, way, car_model, region2
ORDER BY date_type, region2, car_model, way
"""

In [ ]:
df_car_margin = pd.io.gbq.read_gbq(
    query=query_car_margin,
    project_id="socar-data"
)

In [ ]:
# Plotly Express를 사용하여 막대 그래프 생성
fig = px.histogram(df_car_margin, x='car_model', y='use_margin',
                   color='way', barmode='group',
                   hover_data=['use_margin', 'use_time'],
             title='차량별 건당 공헌이익')

# 그래프 표시
fig.show()

In [ ]:
query_car_margin = """
WITH base AS (
  SELECT
      extract(year from date) as year
      , extract(month from date) as month
      , r.isoweek as isoweek
      , r.date as date
      , CASE WHEN  extract(month FROM date) IN (1, 2, 3) THEN '1Q'
            WHEN  extract(month FROM date) IN (4, 5, 6) THEN '2Q'
            WHEN  extract(month FROM date) IN (7, 8, 9) THEN '3Q' ELSE '4Q' END as q
      , r.way as way
      , case when r.way IN ("d2d_round", "d2d_oneway") then 'd2d'
            when r.way = "z2d_oneway" then "z2d_oneway" else 'round' end as way_g
      , case when date BETWEEN '2023-11-06' AND '2023-12-07' THEN 'Before'
             WHEN date BETWEEN '2023-12-08' ANd '2024-01-08' THEN 'After' END as date_type

      , r.region1
      , r.region2

      , cl.car_model

      , count(r.reservation_id) as nuse -- 예약건수
      , sum(r.utime) as utime -- 이용시간

      , sum(revenue) as revenue
      , sum(_rev_d2d) as _rev_d2d -- 부름매출(부름요금 + 편도요금)

      , sum(contribution_margin) as margin

      , sum(__rev_d2d_fee) as d2d_fee -- 부름 요금
      , sum(__rev_d2d_return_fee) as d2d_return_fee -- 부름편도 요금

      , sum(parking_cost_d2d) as parking_cost_d2d --주차료 _부름
      , sum(transport_cost_d2d) as transport_cost_d2d --운반비(부름)

  FROM `socar-data.soda_store.reservation_v2` r
  LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
  LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
  LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
  LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
  WHERE 1=1 AND date >= '2023-11-06'
            AND r.member_imaginary IN (0,9)
            AND region1 = '제주특별자치도'
          --  AND r.zone_id NOT IN (105, 9890, 17209)
  group by isoweek, date, way, way_g, month, region1, region2, car_model, date_type
  -- order by isoweek, date, way, way_g, month, region1, region2, car_model
)

SELECT
  date_type,
  way_g, car_model,
  region2,
  sum(nuse) as nuse,
  sum(utime)/sum(nuse) as use_time,
  sum(margin) as margin,
  sum(margin)/sum(nuse) as use_margin,

  sum(transport_cost_d2d) as transport_cost_d2d,
  sum(transport_cost_d2d)/sum(nuse) as use_transport_cost
FROM base
WHERE date_type = "After"
-- AND way IN ('d2d_round', 'd2d_oneway')
GROUP BY date_type, way_g, car_model, region2
ORDER BY date_type, region2, car_model, way_g
"""

In [ ]:
df_car_margin2 = pd.io.gbq.read_gbq(
    query=query_car_margin,
    project_id="socar-data"
)

In [ ]:
# Plotly Express를 사용하여 막대 그래프 생성
fig = px.histogram(df_car_margin2, x='car_model', y='use_margin',
                   color='way_g', barmode='group',
                   hover_data=['use_margin', 'use_time'],
             title='차량별 건당 공헌이익')

# 그래프 표시
fig.show()

In [ ]:
query_dtd = """
SELECT
scParentId as rid,
r.way,

-- hr.startlng, hr.startlat,
rv.dtod_start_lng as start_lng,
rv.dtod_start_lat as start_lat,
CASE WHEN hr.startaddress1 LIKE '%제주시%' THEN  '제주시'
     WHEN hr.startaddress1 LIKE '%서귀포시%' THEN  '서귀포시' END as start_city,

-- hr.endlng, hr.endlat,
rv.dtod_end_lng as end_lng,
rv.dtod_end_lat as end_lat,
CASE WHEN hr.endAddress1 LIKE '%제주시%' THEN  '제주시'
     WHEN hr.endAddress1 LIKE '%서귀포시%' THEN  '서귀포시' END as end_city,

FROM `socar-data.handler_replica.socarhandler_reservation` hr
LEFT JOIN `tianjin_replica.reservation_info` r ON hr.scParentId = r.id
LEFT JOIN `tianjin_replica.carzone_info` z ON r.zone_id = z.id
LEFT JOIN `soda_store.reservation_v2` rv ON hr.scParentId = rv.reservation_id
WHERE TIMESTAMP_TRUNC(created, DAY) BETWEEN '2023-12-08' AND '2024-01-08'
AND z.region1 = '제주특별자치도'
AND hr.way IN ('d2d_export_h')
AND r.state IN (1, 2, 3)
AND r.member_imaginary IN (0, 9)
ORDER BY rid
"""

In [ ]:
df_dtod = pd.io.gbq.read_gbq(
    query=query_dtd,
    project_id="socar-data"
)

In [ ]:
df_dtod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rid         520 non-null    Int64  
 1   way         520 non-null    object 
 2   start_lng   520 non-null    float64
 3   start_lat   520 non-null    float64
 4   start_city  520 non-null    object 
 5   end_lng     520 non-null    float64
 6   end_lat     520 non-null    float64
 7   end_city    520 non-null    object 
dtypes: Int64(1), float64(4), object(3)
memory usage: 33.1+ KB


In [ ]:
df_dtod[df_dtod['start_city'] == '서귀포시']

,rid,way,start_lng,start_lat,start_city,end_lng,end_lat,end_city
13,62983403,d2d_round,126.407558,33.248222,서귀포시,126.407558,33.248222,서귀포시
17,63108622,d2d_round,126.538130,33.243352,서귀포시,126.538606,33.243826,서귀포시
32,63365526,d2d_round,126.701849,33.273313,서귀포시,126.701849,33.273313,서귀포시
34,63384273,d2d_round,126.509596,33.252125,서귀포시,126.509596,33.252125,서귀포시
41,63465680,d2d_round,126.509596,33.252125,서귀포시,126.509596,33.252125,서귀포시
...,...,...,...,...,...,...,...,...
512,64550813,d2d_oneway,126.508989,33.509104,서귀포시,126.522128,33.517833,제주시
513,64551666,d2d_round,126.556281,33.247894,서귀포시,126.556281,33.247894,서귀포시
514,64556978,d2d_oneway,126.557511,33.249416,서귀포시,126.492828,33.488905,서귀포시
515,64557737,d2d_oneway,126.701849,33.273313,서귀포시,126.513875,33.505898,서귀포시


In [ ]:
df_dtod[df_dtod['start_city'] == '제주시']

,rid,way,start_lng,start_lat,start_city,end_lng,end_lat,end_city
0,61376131,d2d_round,126.497788,33.492988,제주시,126.497788,33.492988,제주시
1,61574985,d2d_round,126.488613,33.485270,제주시,126.488613,33.485270,제주시
2,61577099,d2d_round,126.534144,33.519469,제주시,126.534144,33.519469,제주시
3,62374389,d2d_round,126.481547,33.487558,제주시,126.481547,33.487558,제주시
4,62426399,d2d_round,126.534144,33.519469,제주시,126.534144,33.519469,제주시
...,...,...,...,...,...,...,...,...
506,64539995,d2d_round,126.527636,33.512240,제주시,126.527636,33.512240,제주시
511,64549432,d2d_oneway,126.497784,33.493243,제주시,126.561729,33.246120,제주시
517,64574318,d2d_round,126.796854,33.553281,제주시,126.796854,33.553281,제주시
518,64576753,d2d_oneway,126.511559,33.514888,제주시,126.512005,33.513729,제주시


In [ ]:
df_jeju = df_dtod[df_dtod['start_city'] == '제주시']
df_seo = df_dtod[df_dtod['start_city'] == '서귀포시']

In [ ]:
# Create a map centered around the average of start coordinates
map_center1 = [df_jeju['start_lat'].mean(), df_jeju['start_lng'].mean()]
mymap1 = folium.Map(location=map_center1, zoom_start=12)

In [ ]:
df_jeju[df_jeju['start_city'] == '서귀포시']

,rid,way,start_lng,start_lat,start_city,end_lng,end_lat,end_city


In [ ]:
df_jeju

,rid,way,start_lng,start_lat,start_city,end_lng,end_lat,end_city
0,61376131,d2d_round,126.497788,33.492988,제주시,126.497788,33.492988,제주시
1,61574985,d2d_round,126.488613,33.485270,제주시,126.488613,33.485270,제주시
2,61577099,d2d_round,126.534144,33.519469,제주시,126.534144,33.519469,제주시
3,62374389,d2d_round,126.481547,33.487558,제주시,126.481547,33.487558,제주시
4,62426399,d2d_round,126.534144,33.519469,제주시,126.534144,33.519469,제주시
...,...,...,...,...,...,...,...,...
506,64539995,d2d_round,126.527636,33.512240,제주시,126.527636,33.512240,제주시
511,64549432,d2d_oneway,126.497784,33.493243,제주시,126.561729,33.246120,제주시
517,64574318,d2d_round,126.796854,33.553281,제주시,126.796854,33.553281,제주시
518,64576753,d2d_oneway,126.511559,33.514888,제주시,126.512005,33.513729,제주시


In [ ]:
#  제주시
for index, row in df_jeju.iterrows():
    folium.Marker(location=[row['start_lat'], row['start_lng']], popup=f"Start: {row['way']}",
                  icon=folium.Icon(color='blue')).add_to(mymap1)
    folium.Marker(location=[row['end_lat'], row['end_lng']], popup=f"End: {row['way']}",
                  icon=folium.Icon(color='red')).add_to(mymap1) ## 도착

# Show the map
mymap1

In [ ]:
#  서귀포시
for index, row in df_seo.iterrows():
    folium.Marker(location=[row['start_lat'], row['start_lng']], popup=f"Start: {row['way']}",
                  icon=folium.Icon(color='blue')).add_to(mymap2)
    folium.Marker(location=[row['end_lat'], row['end_lng']], popup=f"End: {row['way']}",
                  icon=folium.Icon(color='red')).add_to(mymap2)

# Show the map
mymap2

In [ ]:
df_jeju_round = df_jeju[df_jeju['way'] == 'd2d_round']
df_jeju_one = df_jeju[df_jeju['way'] == 'd2d_oneway']
df_seo_round = df_seo[df_seo['way'] == 'd2d_round']
df_seo_oneway = df_seo[df_seo['way'] == 'd2d_oneway']

In [ ]:
df_jeju_round

,rid,way,start_lng,start_lat,start_city,end_lng,end_lat,end_city
0,61376131,d2d_round,126.497788,33.492988,제주시,126.497788,33.492988,제주시
1,61574985,d2d_round,126.488613,33.485270,제주시,126.488613,33.485270,제주시
2,61577099,d2d_round,126.534144,33.519469,제주시,126.534144,33.519469,제주시
3,62374389,d2d_round,126.481547,33.487558,제주시,126.481547,33.487558,제주시
4,62426399,d2d_round,126.534144,33.519469,제주시,126.534144,33.519469,제주시
...,...,...,...,...,...,...,...,...
489,64498023,d2d_round,126.319588,33.463724,제주시,126.319588,33.463724,제주시
500,64529424,d2d_round,126.618461,33.535685,제주시,126.618461,33.535685,제주시
501,64530332,d2d_round,126.521233,33.518428,제주시,126.521233,33.518428,제주시
506,64539995,d2d_round,126.527636,33.512240,제주시,126.527636,33.512240,제주시


In [ ]:
# Create a map centered around the average of start coordinates
map_center3 = [df_jeju_round['end_lat'].mean(), df_jeju_round['end_lng'].mean()]
mymap3 = folium.Map(location=map_center3, zoom_start=12)

map_center4 = [df_jeju_one['end_lat'].mean(), df_jeju_one['end_lng'].mean()]
mymap4 = folium.Map(location=map_center4, zoom_start=12)

map_center5 = [df_seo_round['end_lat'].mean(), df_seo_round['end_lng'].mean()]
mymap5 = folium.Map(location=map_center5, zoom_start=12)

map_center6 = [df_seo_oneway['end_lat'].mean(), df_seo_oneway['end_lng'].mean()]
mymap6 = folium.Map(location=map_center6, zoom_start=12)

In [ ]:
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
df_dtod

,region2,rid,way,start_lng,start_lat,end_lng,end_lat
0,서귀포시,63636594,d2d_round,126.408628,33.247145,126.407467,33.248404
1,서귀포시,63615461,d2d_round,126.486502,33.490670,126.486502,33.490670
2,서귀포시,63622964,d2d_round,126.433535,33.247605,126.433535,33.247605
3,제주시,63629838,d2d_round,126.488789,33.485228,126.488789,33.485228
4,서귀포시,63633319,d2d_round,126.481806,33.485725,126.481806,33.485725
...,...,...,...,...,...,...,...
533,서귀포시,64446860,d2d_round,126.488596,33.485078,126.488596,33.485078
534,서귀포시,64556978,d2d_oneway,126.557511,33.249416,126.492828,33.488905
535,서귀포시,64551666,d2d_round,126.556281,33.247894,126.556281,33.247894
536,제주시,64260598,d2d_round,126.529809,33.489682,126.529626,33.489700


In [ ]:
    geolocator = Nominatim(user_agent="reverse_geocoding")
    location = geolocator.reverse((row['start_lat'], row['start_lng']), language='kor')
    address = location.address
    city = address.split(",")[-3].strip()
    address

'신대로5길, 오라삼동, 제주시, 제주특별자치도, 63125, 대한민국'

In [ ]:
# 주소를 적절하게 파싱하여 동(dong)과 시(city) 추출
def parse_address(address):
    elements = address.split(', ')

    # 시(city) 추출
    city = elements[2]

    return city

# 역 지오코딩을 수행하고 도시를 추출하는 함수
def get_city1(row):
    geolocator = Nominatim(user_agent="reverse_geocoding")
    location = geolocator.reverse((row['start_lat'], row['start_lng']), language='kor')

    # 주소 정보가 있는지 확인
    if location and location.address:
        address_parts = location.address.split(",")

        # 리스트에 충분한 요소가 있는지 확인
        if len(address_parts) >= 3:
            # 시 추출
            city = parse_address(location.address)
            return city

    # 도시를 추출할 수 없는 경우 None 반환
    None

# 역 지오코딩을 수행하고 도시를 추출하는 함수
def get_city2(row):
    geolocator = Nominatim(user_agent="reverse_geocoding")
    location = geolocator.reverse((row['end_lat'], row['end_lng']), language='kor')

    # 주소 정보가 있는지 확인
    if location and location.address:
        address_parts = location.address.split(",")

        # 리스트에 충분한 요소가 있는지 확인
        if len(address_parts) >= 3:
            # 시 추출
            city = parse_address(location.address)
            return city

    # 도시를 추출할 수 없는 경우 None 반환
    None

# 예시 데이터프레임에 함수 적용
df_dtod['start_city'] = df_dtod.apply(get_city1, axis=1)
df_dtod['end_city'] = df_dtod.apply(get_city2, axis=1)

In [ ]:
df_dtod

,region2,rid,way,start_lng,start_lat,end_lng,end_lat,start_city,end_city
0,서귀포시,63636594,d2d_round,126.408628,33.247145,126.407467,33.248404,서귀포시,제주특별자치도
1,서귀포시,63615461,d2d_round,126.486502,33.490670,126.486502,33.490670,제주시,제주시
2,서귀포시,63622964,d2d_round,126.433535,33.247605,126.433535,33.247605,서귀포시,서귀포시
3,제주시,63629838,d2d_round,126.488789,33.485228,126.488789,33.485228,제주시,제주시
4,서귀포시,63633319,d2d_round,126.481806,33.485725,126.481806,33.485725,제주특별자치도,제주특별자치도
...,...,...,...,...,...,...,...,...,...
533,서귀포시,64446860,d2d_round,126.488596,33.485078,126.488596,33.485078,제주시,제주시
534,서귀포시,64556978,d2d_oneway,126.557511,33.249416,126.492828,33.488905,서문로,제주시
535,서귀포시,64551666,d2d_round,126.556281,33.247894,126.556281,33.247894,서문로,서문로
536,제주시,64260598,d2d_round,126.529809,33.489682,126.529626,33.489700,제주시,제주시
